# Projeto 1 - Ciência dos Dados

Nome: Enricco Gemha

Nome: Marcelo Rabello Barranco

Nome: Rafael Coca Leventhal

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [46]:
%%capture

#Instalando o tweepy
!pip install emojis

In [47]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emojis

In [48]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\enric\Documents\GitHub\projeto1-cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [49]:
filename = 'iphone.xlsx'

In [50]:
train = pd.read_excel(filename)
train

,Treinamento,RELEVÂNCIA
0,- pego meu iphone 15 pro max 6g\n- ligo pra mi...,0
1,@venusbillie ai fica tao mais bonito no iphone...,4
2,mas tambem quem compra iphone 6 em 2021 tem ma...,4
3,"feliz pra crl , pegar meu iphone agr 😅",4
4,@tatielly_keys vc tem iphone?,0
...,...,...
745,minha mãe quer comprar o iphone 12 pra ela e n...,4
746,nunca precisei mexer no meu iphone se antigo p...,0
747,era só um carregador de iphone 😪,0
748,"tô doida com essa película privacidadeee, vi o...",3


In [51]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,RELEVÂNCIA
0,minha mãe podia me dá um iphone só de pena no ...,4
1,qria fazer a trend da tv alem de ser pequena n...,4
2,papo de q o whatsapp de quem tem iphone vai pa...,3
3,iphone 13 vai esperar mais um pouco pq a gata ...,4
4,alguém sabe se quando quebra a tela do iphone ...,3


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Produto: iPhone (dispensa descrições)

* MUITO IRRELEVANTE: falando de outro assunto, não envolve o iPhone, pessoas que não sabem tweetar ou se comunicar (Ex: só colocou uma hashtag)
* IRRELEVANTE: anúncios de venda (EX: venha comprar no magalu)
* NEUTRO: piada sobre o iPhone (EX: iPhone é o Corsa em miniatura kkkkkk)
* RELEVANTE: comentário indireto relacionado ao iPhone (EX: meu professor de ciências passou 30 minutos só falando do novo iPhone dele)
* MUITO RELEVANTE: falando objetivamente do iPhone, tipo opinião, dúvida ou desejo de comprar (EX: iPhone 13 vai esperar um pouco para chegar em minhas mãos)

- MUITO IRRELEVANTE = 0
- IRRELEVANTE = 1
- NEUTRO = 2
- RELEVANTE = 3
- MUITO RELEVANTE = 4

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [52]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = r'[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.    
    text = re.sub(punctuation, '', text)
    text = emojis.decode(text)
    text = re.sub(r'(:\w+:)', r' \g<1> ', text)

    return text

In [53]:
train['Treinamento'] = train['Treinamento'].apply(cleanup)

In [54]:
train

,Treinamento,RELEVÂNCIA
0,pego meu iphone 15 pro max 6g\n ligo pra minh...,0
1,@venusbillie ai fica tao mais bonito no iphone...,4
2,mas tambem quem compra iphone 6 em 2021 tem ma...,4
3,feliz pra crl pegar meu iphone agr :sweat_sm...,4
4,@tatielly_keys vc tem iphone,0
...,...,...
745,minha mãe quer comprar o iphone 12 pra ela e n...,4
746,nunca precisei mexer no meu iphone se antigo p...,0
747,era só um carregador de iphone :sleepy:,0
748,tô doida com essa película privacidadeee vi o ...,3


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**